In [ ]:
import sympy as sm
import sympy.physics.mechanics as me
import numpy as np

from scipy.integrate import solve_ivp
from matplotlib import animation
from IPython.display import HTML
import matplotlib
from matplotlib import patches
import matplotlib.pyplot as plt
%matplotlib inline
import time
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib.ticker import FuncFormatter

This creates a decorator to test functions for usage of CPU time, line by line.\
To see the results, this line: *profiler.print_stats()* must be added. 

In [ ]:
from line_profiler import LineProfiler

profiler = LineProfiler()

def profile(func):
    def inner(*args, **kwargs):
        profiler.add_function(func)
        profiler.enable_by_count()
        return func(*args, **kwargs)
    return inner

I want to model **Berennan's Monorail** as shown in this video, given to me by JM.\
https://www.youtube.com/watch?v=kUYzuAJeg3M

The train is a cuboidal shaped rigid $\left( 2a \times 2b \times 2c \right) $ body TR, with mass $m_T$, moments of inertia $iXX_T. iYY_T, iZZ_T$, body fixed frame AT and center of mass $Dmc_T$ at $c$ above the horizontal X / Y plane.\
Two spinning discs, the gyroscopes, $G_i, i \in {1, 2}$ with same mass $m_G$ and moments of inertia $iXX_G. iYY_G, iZZ_G$, body fixed frames $AG_i$ and center of masses $Dmc_{G_i}$ are mounted at at height $H_2$ above, and $L$ to the left / right (that is in AT.y direction) of, $Dmc_T$.\
As stated in trhe video, the discs spin in opposite directions, and the rotations around their vertical axes are also opposite. (Meaning, they are only parallel if they are parallel to the train)

Forces of randomly moving passenger / other disturbances are modeled by a random force in the AT.y direction, acting on $Dmc_T$\
I model travelling around a curve by setting $\frac{d}{dt} q_{T_3} \ne 0$, and adding a cetrifugal force $F_C \sim \left( \frac{d}{dt} q_{T_3} \right)^2$ acting on $Dmc_T$.

**Note**\
The video gave me the inpression, that to stabilize the train while running straight, no external moment was required.\
I did not manage to model this, and I see no reason, which this should work: When the train tilts about the X1 axis, and the gyroscope spins about the X2 axis, it tries to make the train spin around the X3 axis. This is not allowed as the train is on rails.\
I have to apply a moment on the spinning giscs around its X3 axis. I make the moment proportional to $\phi$ and to $\dfrac{d}{dt} \phi$, where $\phi$ is the rotation of the train around its X1 axis. The size of the factors are set by trial and error.\
This balances the nonorail well, unless the disturbances are too large, in which case the numerical integration fails.

**Variables**

- $N$: Inertial frame
- $A_T$: body fixed frame of the train
- $A_{G_1}, A_{G_2}$: frames fixed to the gyroscopes
- $O$: point fixed in N
- $Dmc_T, Dmc_{G_1}, Dmc_{G_2}$: centers of gravity

- $q_{T_x}, q_{T_z}, u_{T_x}, u_{T_z}$: generalized coordinates / speeds of the train
- $q_{G_y}, q_{G_z}, u_{G_x}, u_{G_z}$: generalized coordinates / speeds of the gyroscope

- $R_1$: radius of the curve. I do not model the train to 'physically' move.
- $L, H_2$: locations of the $Dmc_{G_i} w.r.t. $Dmc_T$
- $force_1$: will hold the random force
- $K_2$: strength of the random force
- $K_{\phi}, K_{\frac{d}{dt} \phi}$: prportionality factors for the moment
 

In [ ]:
# set up the geometry of the monorail
zeit = time.time()

t = me.dynamicsymbols._t

N, AT, AG1, AG2                                   = sm.symbols('N, AT, AG1, AG2', cls=me.ReferenceFrame)
O, DmcT, DmcG1, DmcG2                             = sm.symbols('O, DmcT, DmcG1, DmcG2', cls=me.Point)

qTx, qTz, qGy, qGz = me.dynamicsymbols('qTx, qTz, qGy, qGz')
uTx, uTz, uGy, uGz = me.dynamicsymbols('uTx, uTz, uGy, uGz')   

mT, mG, g, a, b, c, r, Kphidt, Kphi, K2, R1, H2, L, force1 = sm.symbols('mT, mG, g, a, b, c, r, Kphidt, Kphi, K2, R1, H2, L, force1')

O.set_vel(N, 0)

rot  = []                                                     # needed for the kinematic equations  
rot1 = []                                                     # dto.


AT.orient_body_fixed(N, (qTz, qTx, 0.), ('zxy'))
rot.append(AT.ang_vel_in(N))
AT.set_ang_vel(N, uTx*AT.x + uTz*AT.z)
rot1.append(AT.ang_vel_in(N))

AG1.orient_body_fixed(AT, (qGz, qGy, 0), ('zyx'))
rot.append(AG1.ang_vel_in(N))
AG1.set_ang_vel(AT, uGy*AG1.y + uGz*AG1.z)
rot1.append(AG1.ang_vel_in(N))

AG2.orient_body_fixed(AT, (-qGz, -qGy, 0), ('zyx'))
AG2.set_ang_vel(AT, -uGy*AG2.y -uGz*AG2.z)

DmcT.set_pos(O, c*AT.z)
DmcT.v2pt_theory(O, N, AT)

DmcG1.set_pos(DmcT, L*AT.y + H2*AT.z)
DmcG1.v2pt_theory(DmcT, N, AT)

DmcG2.set_pos(DmcT, -L*AT.y + H2*AT.z)
DmcG2.v2pt_theory(DmcT, N, AT)

**Kane's equations**\
The *centripedal* force is: $\bar F_{centripedaö} = m \cdot \omega \times \left( \omega \times \bar R \right)$, where\
$\bar R$ is the vector from the center of the rotation, the center of the curve, to the mass\
$\omega$ is the angula speed of rigid body in question.

Here, I take:\
$\omega = uT_z \cdot AT.z$ anf ignore the contribution of $zT_x$\
$\bar R = -R_1 \cdot \left( \sin(qT_z) \cdot N.x + \cos(qT_z) \cdot N.y \right)$\
and ignore any other effects. As $R_1 >> \vee (a, b, c, L, H_2)$ this seems justified.\
Note, that $ \bar F_{centrifugal} = -\bar F_{centripedal.}$

$Force = force_1 \cdot N.y$ are random forces acting on DmcT. This supposed to model moving passengers, side winds, whatever.\
I model it with a Brownian motion, see below.


In [ ]:
# from the internet. I assumed hat the discs have a width of 1/10 * r
iXXT = 1./12. * mT * 4. * (b**2 + c**2)
iYYT = 1./12. * mT * 4. * (a**2 + c**2)
iZZT = 1./12. * mT * 4. * (a**2 + b**2)

iXXG1 = 1./12. * mG * ((r/10)**2 + 3*r**2)
iYYG1 =                  1./2. * mG * r**2
iZZG1 = 1./12. * mG * ((r/10)**2 + 3*r**2)

# set up the rigid bodies
IT = me.inertia(AT, iXXT, iYYT, iZZT)
Train = me.RigidBody('Train', DmcT, AT, mT, (IT, DmcT))
IG1 = me.inertia(AG1, iXXG1, iYYG1, iZZG1)
Gyro1 = me.RigidBody('Gyro1', DmcG1, AG1, mG, (IG1, DmcG1))
IG2 = me.inertia(AG2, iXXG1, iYYG1, iZZG1)
Gyro2 = me.RigidBody('Gyro2', DmcG2, AG2, mG, (IG2, DmcG2))

BODY = [Train, Gyro1, Gyro2]

# set up the external forces
F_grav   = [(DmcT, -mT*g*N.z), (DmcG1, -mG*g*N.z), (DmcG2, -mG*g*N.z)]
F_torque = [(AG1, (Kphidt * uTx + Kphi * qTx) * AG1.z)]

radius = -R1*(sm.sin(qTx)*N.x + sm.cos(qTz)*N.y)
omega  = uTz*AT.z
Fac_cent = (-omega.cross(omega.cross(radius)))
F_cent = [(DmcT, -mT*Fac_cent), (DmcG1, -mG*Fac_cent), (DmcG2, -mG*Fac_cent)]

F_rand = [(DmcT, force1*AT.y)]

FL = F_grav + F_torque + F_rand + F_cent

# kinematic equations
kd = [me.dot(rot[0] - rot1[0], uv) for uv in (N.x, N.z)] + [me.dot(rot[1] - rot1[1], uv) for uv in (N.y, N.z)]

q_ind = [qTx, qTz, qGy, qGz]
u_ind = [uTx, uTz, uGy, uGz]

#@profile
def KANE():
    for _ in range(1):
        KM = me.KanesMethod(N, q_ind=q_ind, u_ind=u_ind, kd_eqs=kd)
        (fr, frstar) = KM.kanes_equations(BODY, FL)
        MM = KM.mass_matrix_full
        force = KM.forcing_full
        print('force DS', me.find_dynamicsymbols(force))
        print('force free symbols', force.free_symbols)
        print(f'force has {sm.count_ops(force):,} operations, {sm.count_ops(sm.cse(force)):,} operations after cse', '\n')
        print('MM DS', me.find_dynamicsymbols(MM))
        print('MM free symbols', MM.free_symbols)
        print(f'MM has {sm.count_ops(MM):,} operations, {sm.count_ops(sm.cse(MM)):,} operations after cse', '\n')
        return MM, force
    
MM, force = KANE()
#profiler.print_stats()

**Energies**\
I believe, not very meaningful here.

In [ ]:
kin_energie = sum([koerper.kinetic_energy(N) for koerper in BODY])
pot_energie = mT * g * DmcT.pos_from(O).dot(N.z) + mG * g * DmcG1.pos_from(O).dot(N.z) + mG * g * DmcG2.pos_from(O).dot(N.z)

**Lambdification**

In [ ]:
qL = q_ind + u_ind
pL = [mT, mG, g, a, b, c, r, Kphidt, Kphi, K2, R1, H2, L, force1]

force_lam = sm.lambdify(qL + pL, force, cse=True)
MM_lam    = sm.lambdify(qL + pL, MM, cse=True)

kin_lam = sm.lambdify(qL + pL, kin_energie, cse=True)
pot_lam = sm.lambdify(qL + pL, pot_energie, cse=True)

This creates a **Brownian motion**, symmetric if p = 0.5, else with drift. I found this here: https://medium.com/@mlblogging.k/simulating-brownian-motion-and-stock-prices-using-python-17b6b4bd2a1 \
To simulate a force acting on the train in AT.y direction.

In [ ]:
def bm(nsteps, p, stepsize, kraft):
    steps = [ 1*stepsize if np.random.random() < p else -1*stepsize for _ in range(nsteps) ]
    y = np.cumsum(steps) / nsteps * kraft

    return list(y)

**Numerical integration**

In [ ]:
start1 = time.time()
#========================================
# set parameters
#=======================================
    
mT1 = 22.
mG1 = 3.5
g1 = 9.81

a1 = 10
b1 = 1
c1 = 1.5

r1 = 0.5

Kphidt1 = -1000.        # proportionality of the speed uTx, for the torque applied to the gyroscope
Kphi1    = 0.           # proportionality of the angle qTx, for the torque applied to the gyroscope. Does not seem to work


K21 = 1000.             # factor for the strength of the Brownian motion type force applied to the center of gravity of the train.

R11 = 100.
H21 = 1.
L1 = 1.
force11 = 1.
force21 = 1.

qTx1 = 0.
qTz1 = 0.
qGy1 = 0.
qGz1 = 0.

uTx1 = 0.1
uTz1 = 0.
uGy1 = 2*np.pi * 3500. / 60.                # as per the video, the discs rotate at 3500 rpm
uGz1 = 0.

intervall = 15.
schritte = 500

z1, z2, z3, z4 = 1., 2., 7., 11.            # start / stop times of curves
if not (0 <= z1 < z2 < z3 < z4 <= intervall):
    print('z_i may be invalid')


# create the wiener process to simulate a force acting on DmcT
nstep = 100000
wiener = bm(nstep, 0.5, 1, K21)
zaehler = -1

def gradient(t, y, args):
    global zaehler

    zaehler += 1
    zaehler1 = zaehler % len(wiener)    # if len(wiener) < nfev, then the wiener process is repeated.
    args[-1] = wiener[zaehler1]

    if z1 < t < z2 or z3 < t < z4:      # entering the curve / leaving the curve
        y[5] = np.sqrt(0.001)
    else:
        y[5] = 0.
    
    sol = np.linalg.solve(MM_lam(*y, *args), force_lam(*y, *args))
    return np.array(sol).T[0]


times = np.linspace(0., intervall, int(schritte*intervall))
t_span = (0., intervall)

pL_vals = [mT1, mG1, g1, a1, b1, c1, r1, Kphidt1, Kphi1, K21, R11, H21, L1, force11]
y0 = [qTx1, qTz1, qGy1, qGz1]  + [uTx1, uTz1, uGy1, uGz1] + []

resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), atol=1.e-10, rtol=1.e-10) #, method='Radau') 
resultat = resultat1.y.T
print('Shape of result: ', resultat.shape)
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ' the message is: ', resultat1.message)
print(f'to simulate {intervall:.3f} sec, the routine made {resultat1.nfev:,} function calls and it took {(time.time() - start1):.3f} CPU - sec  ')

Plot the generalized coordinates

In [ ]:
def thousands_formatter(x, pos):
    return "{:,}".format(int(x))


bezeichnung = [str(i) for i in qL]
fig, (ax1, ax1a, ax2, ax3) = plt.subplots(4, 1, figsize=(10, 22), sharex=False)

ax1.plot(times, np.rad2deg(resultat[:, 0]), color='green')  # Changed plot color to green
ax1.set_ylabel('degree [°]')
ax1.set_title(f'Deviation of the train from the (ideal, $\phi = 0$) vertical position \n The smaller the deviation, the better the train is stabilized')

fliehkraft = ((mT + 2*mG) * Fac_cent).magnitude()
fliehkraft_lam = sm.lambdify(qL + pL, fliehkraft, cse=True)
fliehkraft_np = fliehkraft_lam(*[resultat[:, j]for j in range(resultat.shape[1])], *pL_vals) * 1000
wiener1 = []
reduction = int(max(1, len(wiener)/len(times)))
for i in range(len(wiener)):
    if i % reduction == 0:
        wiener1.append(wiener[i])
        if len(wiener1) >= len(times):
            break
print(len(wiener1))
ax1a.plot(times[: len(wiener1)], [wiener1[i] * K21 for i in range(len(wiener1))], label='random force')
ax1a.plot(times, fliehkraft_np, label='Centrifugal Force')
ax1a.set_title('Forces acting on the center of gravity of the train')
ax1a.set_ylabel('force [N]')
ax1a.yaxis.set_major_formatter(FuncFormatter(thousands_formatter))

ax1a.legend();

ax2.plot(times, np.rad2deg(resultat[:, 3]))
ax2.set_ylabel('degree [°]')
ax2.set_title('Angle of gyroscope around its vertical axis')

iXXT1 = 1./12. * mT1 * 4. * (b1**2 + c1**2)
ii    = iXXT1 * uTx1
einheit ='$\dfrac{\mathrm{kg} \cdot \mathrm{m}^2}{\mathrm{sec}^2}$'
ax3.plot(times, resultat[:, 4] * Kphidt1 *1000. + resultat[:, 0] * Kphi1 * 1000, color='green') 
ax3.set_ylabel('torque [Nm]')
ax3.set_xlabel('time [s]')
ax3.set_title(f'Torque applied to each gyroscope. Initial disturbing impulse {1000*ii:,.0f} {einheit}');
ax3.yaxis.set_major_formatter(FuncFormatter(thousands_formatter))

I olny plot the **kinetic energy**, as the potential energy is $\approx$ constant\
I do not think, this is very elluminating.

In [ ]:
kin_np = kin_lam(*[resultat[:, j]for j in range(resultat.shape[1])], *pL_vals)
pot_np = pot_lam(*[resultat[:, j]for j in range(resultat.shape[1])], *pL_vals)
total_np = kin_np + pot_np

fig, ax4 = plt.subplots(1, 1, figsize=(10, 5), sharex=False)
ax4.plot(times, kin_np, color='red', label='kinetic energy')
#ax4.plot(times, pot_np, color='blue', label='potential energy')
#ax4.plot(times, total_np, color='black', label='total energy')
ax4.set_ylabel('energy [kJ]')
ax4.set_title('Kinetic, potential and total energy of the system')
ax4.yaxis.set_major_formatter(FuncFormatter(thousands_formatter))
ax4.legend();
